## Step by step api's methods description

In [1]:
import os
import re
import pandas as pd
import numpy as np
import qiime2
from qiime2 import Artifact
from qiime2.plugins import metadata, feature_table, diversity, emperor
from q2_emperor import plot, procrustes_plot, biplot, generic_plot
from scipy.spatial.distance import squareform, pdist
import skbio

In [5]:
!ls ..

ClusterApp  boot  dev  home  lib64  mnt  proc  run   srv  tmp  var
bin	    data  etc  lib   media  opt  root  sbin  sys  usr


In [6]:
!pwd

/ClusterApp


In [7]:
!ls

 Dockerfile				    README.md	  docker-compose.yml
 Makefile				    __pycache__   notebooks
'Quant_table_mzmine.xlsx - Planilha1.csv'   api		  run_server.sh


In [11]:
def reformatTable(feat_table):
    """
    This Method Prepare The Meta And Feat Table To qiime2 and Return the pcoa object
    """
    last_attr = feat_table.columns[feat_table.columns.str.contains('ATTRIBUTE')][-1]
    plast_attr = feat_table.columns.get_loc(last_attr)+1
    meta = feat_table[feat_table.columns[:plast_attr]]
    meta.columns = meta.columns.str.replace('Filename', 'filename')
    meta.filename+' Peak area'
    meta.fillna('empty', inplace=True)
    meta = meta.astype(str)
    feat = feat_table[feat_table.columns[plast_attr:]].T
    feat.columns = meta.filename+' Peak area'
    feat_tmp = pd.DataFrame(feat.index)
    feat_tmp.reset_index(inplace=True)
    feat_tmp = pd.DataFrame(feat_tmp[0].apply(lambda a: a.split("_")[:2]).tolist())
    feat_tmp.reset_index(inplace=True)

    feat_tmp.columns = ['row ID', 'row m/z', 'row retention time']
    feat = pd.concat([feat_tmp, feat.reset_index(drop=True)], axis=1)
    return feat,meta

In [74]:
df = pd.read_csv('Quant_table_mzmine.xlsx - Planilha1.csv', decimal=',')
df.head()

,filename,ATTRIBUTE_Step,ATTRIBUTE_Method,ATTRIBUTE_Sample,1_1,1_2,1_3,1_4,1_5,1_6,...,1_378,1_379,1_380,1_381,1_382,1_383,1_384,1_385,1_386,1_387
0,file_1,Blank,Blank,Blank,0.0000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,file_2,Blank,Blank,Blank,0.0000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,file_3,Blank,Blank,Blank,0.0000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,file_4,Blank,Blank,Blank,0.0000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,file_5,Blank,Blank,Blank,894.4632,13074.168,1953.7959,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
df['filename'] = df['filename']+'.mzML'

In [76]:
feat,meta = reformatTable(df)

/tmp/ipykernel_447/443562344.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta.fillna('empty', inplace=True)


In [77]:
feat.head()

,row ID,row m/z,row retention time,file_1.mzML Peak area,file_2.mzML Peak area,file_3.mzML Peak area,file_4.mzML Peak area,file_5.mzML Peak area,file_6.mzML Peak area,file_7.mzML Peak area,...,file_20.mzML Peak area,file_21.mzML Peak area,file_22.mzML Peak area,file_23.mzML Peak area,file_24.mzML Peak area,file_25.mzML Peak area,file_26.mzML Peak area,file_27.mzML Peak area,file_28.mzML Peak area,file_29.mzML Peak area
0,0,1,1,0.0,0.0,0.0,0.0,894.4632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2,0.0,0.0,0.0,0.0,13074.1680,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,3,0.0,0.0,0.0,0.0,1953.7959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1,4,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1,5,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
meta.head()

,filename,ATTRIBUTE_Step,ATTRIBUTE_Method,ATTRIBUTE_Sample
0,file_1.mzML,Blank,Blank,Blank
1,file_2.mzML,Blank,Blank,Blank
2,file_3.mzML,Blank,Blank,Blank
3,file_4.mzML,Blank,Blank,Blank
4,file_5.mzML,Blank,Blank,Blank


In [18]:
from api.src.utils.utils import qiime2PCoA

In [67]:
def qiime2PCoA(sample_metadata, df, out_dir, metric):
    sample_metadata.rename(index=str, columns={"filename": "#SampleID"},
                           inplace=True)
    sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')

    sample_metadata.index = sample_metadata['#SampleID']
    sample_metadata.drop(['#SampleID'], axis=1, inplace=True)
    qsample_metadata = qiime2.metadata.Metadata(sample_metadata)

    df2 = df[df.columns[df.columns.str.contains(' Peak area')]]
    df2.columns = [re.sub('(.+\.mzX?ML) .+', '\\1', a) for a in df2.columns]
    df2.index = df['row ID'].astype(str)
    df2 = df2.T

    df2.fillna(0, inplace=True)
    zero_proportion = (df2 == 0).sum().sum() / df2.size

    if zero_proportion == 1.0: 
        raise ValueError(f"The DataFrame contains too many zero values ({zero_proportion:.2%}). Please check the input data table or try different scaling and normalization methods.")

    dm1 = squareform(pdist(df2, metric=metric))
    dm1 = skbio.DistanceMatrix(dm1, ids=df2.index.tolist())
    dm1 = Artifact.import_data("DistanceMatrix", dm1)
    pcoa = diversity.methods.pcoa(dm1)
    emperor_plot = emperor.visualizers.plot(pcoa.pcoa, qsample_metadata)

    if '.qzv' in out_dir:
        emperor_plot.visualization.save(out_dir)
    else:
        emperor_plot.visualization.export_data(out_dir)
    return emperor_plot

In [68]:
pcoa_obj = qiime2PCoA(meta, feat,
                      out_dir=f'notebooks/teste_winner',
                      metric='euclidean')

/tmp/ipykernel_447/682895115.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')
/opt/conda/envs/qiime2-2023.7/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -8.413838687101675e-05 and the largest is 337925192482.6943.
  warn(


In [73]:
pcoa_obj = qiime2PCoA(meta, feat,
                      out_dir=f'notebooks/teste_winner_canberra',
                      metric='canberra')

/tmp/ipykernel_447/682895115.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')
/opt/conda/envs/qiime2-2023.7/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -11427.271588377378 and the largest is 118235.0526386743.
  warn(


In [79]:
pcoa_obj = qiime2PCoA(meta, feat,
                      out_dir=f'notebooks/teste_winner_jaccard',
                      metric='jaccard')

/tmp/ipykernel_447/682895115.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')
